In [7]:
!cp .devcontainer/kaggle.json ~/.config/kaggle/kaggle.json

In [60]:
!pip install imageio[ffmpeg]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 839.6 kB/s eta 0:00:00m eta 0:00:010:00:02


In [27]:
kagglehub.whoami()

Kaggle credentials successfully validated.


{'username': 'krishnwin'}

In [36]:
import kagglehub
path = kagglehub.model_download("krishnawin/timesformer_finetuned_workout/pyTorch/2")

100%|██████████████████████████████████████████████| 1.26k/1.26k [00:00<00:00, 1.55MB/s]


100%|████████████████████████████████████████████████| 463M/463M [01:40<00:00, 4.82MB/s]


In [1]:

import torch
from transformers import  TimesformerConfig, TimesformerForVideoClassification,AutoImageProcessor
import imageio
import cv2
import numpy as np
import os
import time

In [37]:
model = TimesformerForVideoClassification.from_pretrained(
    path,
    trust_remote_code=True  # if needed for custom heads
)
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

TimesformerForVideoClassification(
  (timesformer): TimesformerModel(
    (embeddings): TimesformerEmbeddings(
      (patch_embeddings): TimesformerPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.2, inplace=False)
      (time_drop): Dropout(p=0.2, inplace=False)
    )
    (encoder): TimesformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x TimesformerLayer(
          (drop_path): Identity()
          (attention): TimeSformerAttention(
            (attention): TimesformerSelfAttention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (attn_drop): Dropout(p=0.2, inplace=False)
            )
            (output): TimesformerSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
          )
          (intermediate): TimesformerIntermediate(
            (dense

In [44]:
image_processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")


preprocessor_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [53]:
# Preprocess Videos for ViViT
def preprocess_video(video_path, image_processor, num_frames=8):
    video_reader = imageio.get_reader(video_path, "ffmpeg")
    frames = []
    for i, frame in enumerate(video_reader):
        frames.append(frame)
    video_reader.close()
    # Sample frames uniformly
    indices = np.linspace(0, len(frames) - 1, num_frames, dtype=int)
    frames = [frames[i] for i in indices]
    # Use the feature extractor for preprocessing
    inputs = image_processor(frames, return_tensors="pt")
    return inputs


In [62]:
path = "./notebooks/data/yt-vtt/fitclass/"
for video_file in os.listdir("./notebooks/data/yt-vtt/fitclass/"):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(path, video_file)
        print(f"Processing {video_path}...")
        
        # Preprocess the video
        inputs = preprocess_video(video_path, image_processor)
        
        # Move inputs to the appropriate device
        inputs = {k: v.to("cuda" if torch.cuda.is_available() else "cpu") for k, v in inputs.items()}
        
        # Perform inference
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get predictions
        logits = outputs.logits
        #probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_class_idx = logits.argmax(-1).item()
        # Print results
        print(f"Predictions for {video_file}: {model.config.id2label[predicted_class_idx]}")

Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_1924_2947.mp4...
Predictions for converted_-cAGR3GiZIU_1924_2947.mp4: hip_thrust
Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_12774_12997.mp4...
Predictions for converted_-cAGR3GiZIU_12774_12997.mp4: crunch
Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_14724_15235.mp4...
Predictions for converted_-cAGR3GiZIU_14724_15235.mp4: plank_push_up
Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_18861_19052.mp4...
Predictions for converted_-cAGR3GiZIU_18861_19052.mp4: crunch
Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_5462_5860.mp4...
Predictions for converted_-cAGR3GiZIU_5462_5860.mp4: hip_thrust
Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_4157_5461.mp4...
Predictions for converted_-cAGR3GiZIU_4157_5461.mp4: hip_thrust
Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_10572_11595.mp4...
Predictions for converted_

In [57]:
os.listdir("./")

['.DS_Store',
 'drivers',
 'run.ipynb',
 'setup',
 'utils',
 'models',
 '.devcontainer',
 '.gitignore',
 '.env',
 'yml',
 '.ipynb_checkpoints',
 '.git',
 '.vscode',
 'notebooks',
 '.conda']

In [3]:
%pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 8.7 MB/s eta 0:00:008.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [boto3]━━━━━ 3/4 [boto3]re]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ray
ray.init()

2025-07-22 05:47:49,360	WARNING services.py:2152 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.14gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-07-22 05:47:50,980	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.11
Ray version:,2.47.1
Dashboard:,http://127.0.0.1:8265


(ProxyActor pid=89085) INFO 2025-07-22 05:48:56,035 proxy 172.17.0.2 -- Proxy starting on node a360bc2b9a5ce20b0ed8e228e7a002fbd6dbc66ea20148141b4244e4 (HTTP port: 8000).
(ProxyActor pid=89085) INFO 2025-07-22 05:48:56,187 proxy 172.17.0.2 -- Got updated endpoints: {}.
(ServeController pid=89083) INFO 2025-07-22 05:49:33,917 controller 89083 -- Deploying new version of Deployment(name='ActionDetectorClass', app='default') (initial target replicas: 1).
(ServeController pid=89083) INFO 2025-07-22 05:49:33,921 controller 89083 -- Deploying new version of Deployment(name='RepCounterClass', app='default') (initial target replicas: 1).
(ServeController pid=89083) INFO 2025-07-22 05:49:33,924 controller 89083 -- Deploying new version of Deployment(name='Ingress', app='default') (initial target replicas: 1).
(ServeController pid=89083) INFO 2025-07-22 05:49:37,881 controller 89083 -- Adding 1 replica to Deployment(name='ActionDetectorClass', app='default').
(ServeController pid=89083) INFO 202

(ServeReplica:default:RepCounterClass pid=89086) /workspaces/visionmlstudio/serve/


(ServeReplica:default:RepCounterClass pid=89086) Using cache found in /home/jovyan/.cache/torch/hub/huggingface_pytorch-image-models_main
(ServeController pid=89083) WARNING 2025-07-22 05:50:11,427 controller 89083 -- Deployment 'ActionDetectorClass' in application 'default' has 1 replicas that have taken more than 30s to be scheduled. This may be due to waiting for the cluster to auto-scale or for a runtime environment to be installed. Resources required for each replica: {"CPU": 1}, total resources available: {"CPU": 13.0}. Use `ray status` for more details.


(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=89082) Received frame 1 from client
(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=89082) Received frame 2 from client
(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=89082) Received frame 3 from client
(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=89082) Received frame 4 from client
(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=89082) Received frame 5 from client
(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=89082) Received frame 6 from client
(ServeReplica:default:Ingress pid=89082) Received frame (230400,) from client
(ServeReplica:default:Ingress pid=

(ServeReplica:default:Ingress pid=89082) INFO 2025-07-22 05:51:27,348 default_Ingress c48kqxab c67a66b5-53c6-4d3c-b6d9-7e0455d71d84 -- Started <ray.serve._private.router.SharedRouterLongPollClient object at 0x7f83ca163290>.
(ServeReplica:default:ActionDetectorClass pid=89102) /opt/conda/lib/python3.12/site-packages/ray/serve/_private/replica.py:1365: UserWarning: Calling sync method '__call__' directly on the asyncio loop. In a future version, sync methods will be run in a threadpool by default. Ensure your sync methods are thread safe or keep the existing behavior by making them `async def`. Opt into the new behavior by setting RAY_SERVE_RUN_SYNC_IN_THREADPOOL=1.
(ServeReplica:default:ActionDetectorClass pid=89102)   warnings.warn(
(ServeReplica:default:ActionDetectorClass pid=89102) ERROR 2025-07-22 05:51:27,603 default_ActionDetectorClass xf8s7srp c67a66b5-53c6-4d3c-b6d9-7e0455d71d84 -- Request failed.
(ServeReplica:default:ActionDetectorClass pid=89102) Traceback (most recent call 

(ServeReplica:default:Ingress pid=89082) WebSocket error: ray::ServeReplica:default:ActionDetectorClass.handle_request_with_rejection() (pid=89102, ip=172.17.0.2, actor_id=b89a180a864815d97b68b71201000000, repr=<ray.serve._private.replica.ServeReplica:default:ActionDetectorClass object at 0x7f6e8fffe5a0>)
(ServeReplica:default:Ingress pid=89082)     async for result in self._replica_impl.handle_request_with_rejection(
(ServeReplica:default:Ingress pid=89082)   File "/opt/conda/lib/python3.12/site-packages/ray/serve/_private/replica.py", line 685, in handle_request_with_rejection
(ServeReplica:default:Ingress pid=89082)     yield await asyncio.wrap_future(
(ServeReplica:default:Ingress pid=89082)           ^^^^^^^^^^^^^^^^^^^^^^^^^^
(ServeReplica:default:Ingress pid=89082)   File "/opt/conda/lib/python3.12/site-packages/ray/serve/_private/replica.py", line 1672, in call_user_method
(ServeReplica:default:Ingress pid=89082)     result, sync_gen_consumed = await self._call_func_or_gen(
(Se

In [2]:
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

True

In [3]:
%run ./serve/actiondetection_timesformer.py

2025-07-22 05:48:45,099	INFO worker.py:1723 -- Connecting to existing Ray cluster at address: 172.17.0.2:48610...
2025-07-22 05:48:45,103	INFO worker.py:1747 -- Calling ray.init() again after it has already been called.
INFO 2025-07-22 05:48:56,347 serve 88234 -- Started Serve in namespace "serve".
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
INFO 2025-07-22 05:49:12,217 serve 88234 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
INFO 2025-07-22 05:50:27,275 serve 88234 -- Application 'default' is ready at http://127.0.0.1:8000/.


In [ ]:
import base64
import asyncio
import websockets
import cv2
import numpy as np
import os
import imageio
import json

# Fix for "RuntimeError: This event loop is already running" in Jupyter
import nest_asyncio
nest_asyncio.apply()

path = "./notebooks/data/yt-vtt/fitclass/"
for video_file in os.listdir("./notebooks/data/yt-vtt/fitclass/"):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(path, video_file)
        print(f"Processing {video_path}...")
        video_reader = imageio.get_reader(video_path, "ffmpeg")
        frames = []
        for i, frame in enumerate(video_reader):
            frames.append(frame)
        video_reader.close()
        # Sample frames uniformly
        indices = np.linspace(0, len(frames) - 1, 96, dtype=int)
        frames = [frames[i] for i in indices]
        async def send_frames(frames):
            # Encode each frame as JPEG and then base64
            encoded_frames = []
            for frame in frames:
                resized_frame = cv2.resize(frame, (224, 224))
                _, buffer = cv2.imencode('.jpg', resized_frame)
                encoded = base64.b64encode(buffer).decode('utf-8')
                print(f"Encoded frame size: {len(encoded)} bytes")
                encoded_frames.append(encoded)
            async with websockets.connect("ws://127.0.0.1:8000/ws") as websocket:
                response = None
                for encoded in encoded_frames:
                    await websocket.send(  json.dumps({'frame':encoded}))
                    response = await websocket.recv()
                    print(f"Received response: {response}")
                return response

        predicted_class_idx = None
        import websockets

        try:
            response = asyncio.get_event_loop().run_until_complete(send_frames(frames))
        except websockets.exceptions.ConnectionClosedOK:
            # Suppress the normal closure exception
            response = None

        # You can parse response if needed, here we just set predicted_class_idx as an example
        try:
            #print(f"Received response: {response}")
            predicted_class_idx = int(response)
        except Exception:
            predicted_class_idx = 0  # fallback or handle error
        
        print(f"Predictions for {video_file}: {predicted_class_idx}")

Processing ./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_1924_2947.mp4...
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized frame shape: (224, 224, 3)
Resized

KeyboardInterrupt: 

In [6]:
import base64

In [6]:

video_reader = imageio.get_reader('./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_1924_2947.mp4', "ffmpeg")
frames = []
for i, frame in enumerate(video_reader):
            frames.append(frame)
video_reader.close()
        # Sample frames uniformly
#indices = np.linspace(0, len(frames) - 1, 8, dtype=int)
#frames = [frames[i] for i in indices]

In [7]:
encoded_frames = []
for frame in frames:
                resized_frame = cv2.resize(frame, (224, 224))
                _, buffer = cv2.imencode('.jpg', resized_frame)
                encoded = base64.b64encode(buffer).decode('utf-8')
                encoded_frames.append(encoded)

In [14]:
for encoded in encoded_frames:
    decode = base64.b64decode(encoded)
    np_arr = np.frombuffer(decode, np.uint8)
    frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

In [17]:
np_arr.shape

(7750,)

In [36]:
video_reader = imageio.get_reader('./notebooks/data/yt-vtt/fitclass/converted_-cAGR3GiZIU_122_377.mp4', "ffmpeg")
frames = []
for i, frame in enumerate(video_reader):
            frames.append(frame)
video_reader.close()
        # Sample frames uniformly
indices = np.linspace(0, len(frames) - 1, 96, dtype=int)
frames = [frames[i] for i in indices]
async def send_frames(frames):
            # Encode each frame as JPEG and then base64
            encoded_frames = []
            for frame in frames:
                resized_frame = cv2.resize(frame, (224, 224))
                _, buffer = cv2.imencode('.jpg', resized_frame)
                encoded = base64.b64encode(buffer).decode('utf-8')
                encoded_frames.append(encoded)
            async with websockets.connect("ws://127.0.0.1:8000/ws",) as websocket:
                response = None
                i=0
                for encoded in encoded_frames:
                    i += 1
                    print(f"Sending frame:{i}")  # Print first 30 characters for brevity
                    await websocket.send(json.dumps({'frame': encoded}))
                    response = await websocket.recv()
                    print(f"Received response: {response}")
                return response

predicted_class_idx = None
import websockets

try:
            response = asyncio.get_event_loop().run_until_complete(send_frames(frames))
except websockets.exceptions.ConnectionClosedOK:
            # Suppress the normal closure exception
            response = None

        # You can parse response if needed, here we just set predicted_class_idx as an example
try:
            #print(f"Received response: {response}")
            predicted_class_idx = int(response)
except Exception:
            predicted_class_idx = 0  # fallback or handle error
        
print(f"Predictions for {video_file}: {predicted_class_idx}")     

Sending frame:1
Received response: {"status":"ok","counter":1}
Sending frame:2
Received response: {"status":"ok","counter":2}
Sending frame:3
Received response: {"status":"ok","counter":3}
Sending frame:4
Received response: {"status":"ok","counter":4}
Sending frame:5
Received response: {"status":"ok","counter":5}
Sending frame:6
Received response: {"status":"ok","counter":6}
Sending frame:7
Received response: {"status":"ok","counter":7}
Sending frame:8
Received response: {"status":"ok","counter":8}
Sending frame:9
Received response: {"status":"ok","counter":9}
Sending frame:10
Received response: {"status":"ok","counter":10}
Sending frame:11
Received response: {"status":"ok","counter":11}
Sending frame:12
Received response: {"status":"ok","counter":12}
Sending frame:13
Received response: {"status":"ok","counter":13}
Sending frame:14
Received response: {"status":"ok","counter":14}
Sending frame:15
Received response: {"status":"ok","counter":15}
Sending frame:16
Received response: {"statu